In [23]:
import os
import sys
import json
sys.path.append("..")
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import cross_validate, train_test_split, TimeSeriesSplit
import numpy as np
from settings import PROJECT_PATH, CLASSIFICATION_TARGET, REGRESSION_TARGET
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

In [24]:
data = pd.read_parquet(os.path.join(PROJECT_PATH, 'data/interim/transactions_post_feature_engineering.parquet'))

In [25]:
with open ("../preprocessing/features_used/features_name.json", "r") as f :
    features_name = json.load (f)

with open("../preprocessing/features_used/numerical_features.json", "r") as f :
    numerical_features= json.load(f)
print(features_name)
print(numerical_features)

['prix', 'vefa', 'surface_habitable', 'latitude', 'longitude', 'mois_transaction', 'annee_transaction', 'en_dessous_du_marche', 'prix_m2_moyen_mois_precedent', 'nb_transactions_mois_precedent', 'ville_demandee', 'type_batiment_Maison', 'nom_region_Nouvelle-Aquitaine', 'nom_region_Occitanie', "nom_region_Provence-Alpes-Côte d'Azur", 'nom_region_Île-de-France']
['surface_habitable', 'latitude', 'longitude', 'mois_transaction', 'annee_transaction', 'prix_m2_moyen_mois_precedent', 'nb_transactions_mois_precedent']


In [26]:
categorial_features = [col for col in features_name if col not in numerical_features and col not in [REGRESSION_TARGET,CLASSIFICATION_TARGET]]
print(categorial_features)

['vefa', 'ville_demandee', 'type_batiment_Maison', 'nom_region_Nouvelle-Aquitaine', 'nom_region_Occitanie', "nom_region_Provence-Alpes-Côte d'Azur", 'nom_region_Île-de-France']


In [27]:
# Copie du Df d'origine
df = data.copy()
# Séparation du Df en trainset et testset
trainset,testset= train_test_split(df, test_size=0.2, random_state=0)

print(f'Nombre d\'échantillons du trainset: {trainset.shape[0]}')
print(f"proportion de biens en dessous du marché dans le trainset:{round(trainset['en_dessous_du_marche'].value_counts(normalize=True)[1],2)}")
print(f'Nombre d\'échantillons du testset: {testset.shape[0]}')
print(f"proportion de biens en dessous du marché dans le testset:{round(testset['en_dessous_du_marche'].value_counts(normalize=True)[1],2)}")

Nombre d'échantillons du trainset: 374492
proportion de biens en dessous du marché dans le trainset:0.37
Nombre d'échantillons du testset: 93623
proportion de biens en dessous du marché dans le testset:0.37


In [28]:
X_train= trainset.drop([REGRESSION_TARGET,CLASSIFICATION_TARGET],axis=1)
y_train_regression = trainset[REGRESSION_TARGET]
y_train_classification = trainset[CLASSIFICATION_TARGET]

In [29]:
X_test= testset.drop([REGRESSION_TARGET,CLASSIFICATION_TARGET],axis=1)
y_test_regression = testset[REGRESSION_TARGET]
y_test_classification = testset[CLASSIFICATION_TARGET]

In [30]:
def perform_cross_validation(
    X: pd.DataFrame,
    y: pd.Series,
    model,
    cross_val_type,
    scoring_metrics: tuple,
    groupes=None,
):
    """
    Effectue une validation croisée pour un modèle donné et retourne les scores
    d'entraînement et de test, ainsi que le modèle ajusté.

    Paramètres :
    ----------
    X : pd.DataFrame
        Les données d'entrée (features) au format pandas DataFrame.
    y : pd.Series
        Les étiquettes ou cibles au format pandas Series.
    modele : object
        Modèle d'apprentissage supervisé (ex : un modèle sklearn) à valider.
    cross_val_type : int, générateur ou objet de type cross-validation
        La stratégie de validation croisée à utiliser (ex : KFold, StratifiedKFold).
    scoring_metrics : tuple
        Les métriques d'évaluation pour la validation croisée
        (ex : ('accuracy', 'f1')).
    groupes : array-like, optionnel
        Groupes utilisés pour certaines stratégies de validation croisée
        comme GroupKFold.

    Retourne :
    ---------
    scores : dict
        Les scores obtenus pour chaque métrique durant la validation croisée.
    scores_dict : dict
        Moyennes et écarts-types des scores pour chaque métrique.
    modele : object
        Le modèle ajusté sur l'ensemble complet des données (X, y).

    Exemple d'appel :
    -----------------
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> from sklearn.model_selection import KFold
    >>> X = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6]})
    >>> y = pd.Series([0, 1, 0])
    >>> modele = RandomForestClassifier()
    >>> cross_val_type = KFold(n_splits=3)
    >>> scoring_metrics = ('accuracy', 'f1')
    >>> scores, scores_dict, model = effectuer_validation_croisée(X, y, model, cross_val_type, scoring_metrics)
    """
    # Effectuer la validation croisée
    scores = cross_validate(
        model,
        X.values,  # Convertir DataFrame en tableau NumPy
        y.values,  # Convertir Series en tableau NumPy
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=True,
        scoring=scoring_metrics,
        groups=groupes,
    )

    # Calculer les moyennes et écarts-types pour chaque métrique
    scores_dict = {}
    for metrique in scoring_metrics:
        scores_dict["moyenne_train_" + metrique] = np.mean(scores["train_" + metrique])
        scores_dict["ecart_type_train_" + metrique] = np.std(scores["train_" + metrique])
        scores_dict["moyenne_test_" + metrique] = np.mean(scores["test_" + metrique])
        scores_dict["ecart_type_test_" + metrique] = np.std(scores["test_" + metrique])

    # Ajuster le modèle sur l'ensemble des données
    model.fit(X.values, y.values)

    return scores, scores_dict, model

In [31]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [32]:
mlflow.set_tracking_uri(os.path.join(PROJECT_PATH,"mlruns"))

In [33]:
experiment_name = "regression_models"
experiment_tags = {
        "phase": "Model_Comparison",
        "revision_de_donnees": "v1",
        "date_de_construction": "Janvier 2025"
    }

In [34]:
from mlflow import MlflowClient, set_experiment

def configure_mlflow_experiment(experiment_name: str, experiment_tags: dict = None) -> str:
    """
    Configure une expérience MLflow. Crée l'expérience si elle n'existe pas déjà et la définit comme active.

    Args:
        experiment_name (str): Le nom de l'expérience MLflow.
        experiment_tags (dict, optional): Un dictionnaire de tags pour l'expérience. Par défaut, aucun tag.

    Returns:
        str: L'ID de l'expérience configurée.
    """
    client = MlflowClient()
    try:
        # Tente de créer une nouvelle expérience
        experiment_id = client.create_experiment(name=experiment_name, tags=experiment_tags)
        print(f"Nouvelle expérience créée avec l'ID : {experiment_id}")
    except Exception as e:
        # Si l'expérience existe déjà, récupère son ID
        print(f"L'expérience existe déjà : {str(e)}")
        experiment = client.get_experiment_by_name(experiment_name)
        if experiment is None:
            raise ValueError(f"L'expérience {experiment_name} n'a pas pu être récupérée.")
        experiment_id = experiment.experiment_id

    # Définir l'expérience active
    set_experiment(experiment_name)
    print(f"Expérience définie : {experiment_name} (ID : {experiment_id})")

    return experiment_id


In [35]:

def train_and_log_model_with_mlflow(
    model,
    X_train,
    y_train,
    experiment_name,
    run_name,
    model_params,
    tags,
    cross_val_type,
    scoring_metrics,
    groups=None,
):
    """
    Entraîne un modèle, effectue une validation croisée avec `perform_cross_validation`,
    enregistre les résultats et le modèle avec MLflow.

    Paramètres :
    ----------
    model : object
        Modèle d'apprentissage supervisé (ex : un modèle sklearn).
    X_train : pd.DataFrame
        Les données d'entrée pour l'entraînement.
    y_train : pd.Series
        Les étiquettes ou cibles.
    experiment_name : str
        Nom de l'expérience MLflow.
    run_name : str
        Nom du run dans MLflow.
    model_params : dict
        Paramètres du modèle.
    tags : dict
        Tags à ajouter au run MLflow.
    cross_val_type : int, générateur ou objet de type cross-validation
        La stratégie de validation croisée à utiliser (ex : KFold, StratifiedKFold).
    scoring_metrics : tuple
        Les métriques d'évaluation pour la validation croisée
        (ex : ('accuracy', 'f1')).
    groups : array-like, optionnel
        Groupes utilisés pour certaines stratégies de validation croisée
        comme GroupKFold.

    Retourne :
    ---------
    scores_dict : dict
        Moyennes et écarts-types des scores pour chaque métrique.
    """
    import mlflow
    import mlflow.sklearn

    # Définir ou créer l'expérience MLflow
    from mlflow.tracking import MlflowClient

    client = MlflowClient()
    try:
        experiment_id = client.create_experiment(name=experiment_name)
        print(f"Nouvelle expérience créée avec l'ID : {experiment_id}")
    except Exception:
        experiment = client.get_experiment_by_name(experiment_name)
        experiment_id = experiment.experiment_id

    mlflow.set_experiment(experiment_name)

    # Démarrer un nouveau run dans MLflow
    with mlflow.start_run(run_name=run_name):
        # Ajouter les tags au run
        mlflow.set_tags(tags)

        # Effectuer la validation croisée
        scores, scores_dict, trained_model = perform_cross_validation(
            X=X_train,
            y=y_train,
            model=model,
            cross_val_type=cross_val_type,
            scoring_metrics=scoring_metrics,
            groupes=groups,
        )

        # Enregistrer les paramètres du modèle
        for param_name, param_value in model_params.items():
            mlflow.log_param(param_name, param_value)

        # Enregistrer les métriques
        mlflow.log_metrics(scores_dict)



        # Créer la signature du modèle à partir des données
        signature = infer_signature(X_train, y_train)

        # Exemple d'entrée pour le modèle
        input_example = X_train.head(1)
        mlflow.sklearn.log_model(trained_model, "model", input_example=input_example, signature=signature)

        # Enregistrement du DataFrame d'entraînement en tant qu'artefact
        trainset_file = "trainset.csv"
        X_train.to_csv(trainset_file, index=False)
        mlflow.log_artifact(trainset_file)

        # Retourner les scores
        return scores_dict


In [36]:
configure_mlflow_experiment(experiment_name, experiment_tags)

L'expérience existe déjà : Experiment 'regression_models' already exists.
Expérience définie : regression_models (ID : 203965422485053459)


'203965422485053459'

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

# Configuration des paramètres spécifiques au GradientBoostingRegressor
model_params = {
    "n_estimators": 150,
    "learning_rate": 0.07,
    "max_depth": 5,
    "min_samples_split": 4,
    "min_samples_leaf": 2,
    "subsample": 0.9,
    "random_state": 42
}

# Initialisation du modèle
gradient_boosting_model = GradientBoostingRegressor(**model_params)

# Configuration des tags pour MLflow
tags = {
    "Experiment_Type": "models_comparison",
    "Phase": "baseline",
    "Model_Type": "Gradient Boosting Regressor",
    "Task": "Regression",
    "Run_Type": "Tunned",
    "Solver": "Gradient Boosting",
    "Dataset": "trainset",
    "CV_Method": "TimeSeriesSplit",
    "CV_Folds": 5,
}

# Validation croisée
cross_val_type = TimeSeriesSplit(5)
scoring_metrics = ["neg_root_mean_squared_error", "neg_mean_absolute_error", "r2"]

# Appel de la fonction
run_name = "gradient_boosting_regressor_run_tunned1"

scores_dict = train_and_log_model_with_mlflow(
    model=gradient_boosting_model,
    X_train=X_train,
    y_train=y_train_regression,  # Variable cible pour la régression
    experiment_name="regression_models",
    run_name=run_name,
    model_params=model_params,
    tags=tags,
    cross_val_type=cross_val_type,
    scoring_metrics=scoring_metrics,
)

print(f"Scores obtenus avec GradientBoostingRegressor : {scores_dict}")


/mnt/c/Users/mrgxl/Documents/cours françois/exercice python/Machine Learning/prediction_donnees_immobilieres/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/mnt/c/Users/mrgxl/Documents/cours françois/exercice python/Machine Learning/prediction_donnees_immobilieres/.venv/lib/python3.11/site-packages/

Scores obtenus avec GradientBoostingRegressor : {'moyenne_train_neg_root_mean_squared_error': -119221.08607932506, 'ecart_type_train_neg_root_mean_squared_error': 7001.7119029195055, 'moyenne_test_neg_root_mean_squared_error': -150040.32884426275, 'ecart_type_test_neg_root_mean_squared_error': 3880.969165439915, 'moyenne_train_neg_mean_absolute_error': -57342.240645698934, 'ecart_type_train_neg_mean_absolute_error': 1177.5078774203898, 'moyenne_test_neg_mean_absolute_error': -60712.93732574068, 'ecart_type_test_neg_mean_absolute_error': 585.8270718642066, 'moyenne_train_r2': 0.8195443484908977, 'ecart_type_train_r2': 0.01984817669330034, 'moyenne_test_r2': 0.7233680637843023, 'ecart_type_test_r2': 0.010799728863940154}
